# Data Preprocessing


In [ ]:
# Import necessary libraries
import pandas as pd
import re  # Regular expressions
import nltk  # Natural Language Toolkit
import os  # Operating system utilities (for file paths)
from bs4 import BeautifulSoup  # For parsing HTML
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Define the path to the dataset
file_dir = "../DS/Assassin.csv"

# Load the dataset
df = pd.read_csv(f"{file_dir}")

# Display the first 5 rows to understand the data
df.head()

,sender,receiver,date,subject,body,label,urls
0,Robert Elz <kre@munnari.OZ.AU>,Chris Garrigues <cwg-dated-1030377287.06fa6d@D...,"Thu, 22 Aug 2002 18:26:25 +0700",Re: New Sequences Window,"Date: Wed, 21 Aug 2002 10:54:46 -0500 ...",0,1
1,Steve Burt <Steve_Burt@cursor-system.com>,"""'zzzzteana@yahoogroups.com'"" <zzzzteana@yahoo...","Thu, 22 Aug 2002 12:46:18 +0100",[zzzzteana] RE: Alexander,"Martin A posted:\nTassos Papadopoulos, the Gre...",0,1
2,"""Tim Chapman"" <timc@2ubh.com>",zzzzteana <zzzzteana@yahoogroups.com>,"Thu, 22 Aug 2002 13:52:38 +0100",[zzzzteana] Moscow bomber,Man Threatens Explosion In Moscow \n\nThursday...,0,1
3,Monty Solomon <monty@roscom.com>,undisclosed-recipient: ;,"Thu, 22 Aug 2002 09:15:25 -0400",[IRR] Klez: The Virus That Won't Die,Klez: The Virus That Won't Die\n \nAlready the...,0,1
4,Stewart Smith <Stewart.Smith@ee.ed.ac.uk>,zzzzteana@yahoogroups.com,"Thu, 22 Aug 2002 14:38:22 +0100",Re: [zzzzteana] Nothing like mama used to make,"> in adding cream to spaghetti carbonara, whi...",0,1


## Part 1: Basic Text Preprocessing

- Only keep colums for the email's body text and the label
- Drop rows with duplicated or missing values
- Clean email body text by 
    - Extracting plain text
    - Normalizing whitespace
    - Removing emails, URLs, and punctuations
    - Lowercasing words and stripping leading/trailing spaces

In [ ]:
# Keep only the 'body' (email text) and 'label' (spam/not spam)
df = df[['body', 'label']]

### Data Integrity Checks

In [ ]:
# 1. Remove rows with missing values (NaN)
df = df.dropna()
print("Missing values after dropna():")
print(df.isnull().sum())

# 2. Remove rows with duplicated email bodies
df = df.drop_duplicates(subset=['body'])
print(f"\nRemaining duplicates: {df.duplicated(subset=['body']).sum()}")

Missing values after dropna():
body     0
label    0
dtype: int64

Remaining duplicates: 0


In [94]:
def clean_email_body(html_text):
    """
    Cleans raw email text by:
    1. Parsing HTML and extracting plain text.
    2. Normalizing whitespace (newlines, tabs, etc.).
    3. Removing URLs, email addresses, and all non-alphabetic characters.
    4. Lowercasing and stripping final whitespace.
    """
    # 1. Parse HTML and extract plain text
    try:
        soup = BeautifulSoup(html_text, 'html.parser')
        text = soup.get_text()
    except:
        # Handle cases where the body might not be HTML (e.g., just plain text)
        # Ensure the input is treated as a string
        text = str(html_text)

    # 2. Normalize whitespace (replace multiple spaces, newlines, tabs with a single space)
    text = re.sub(r'\s+', ' ', text)

    # 3. Remove noise: URLs, emails, and punctuation
    text = re.sub(r'http\S+', ' ', text)      # Replace URLs with a space
    text = re.sub(r'\S+@\S+', ' ', text)     # Replace emails with a space
    text = re.sub(r'[^a-zA-Z\s]', ' ', text) # Replace non-letters/non-spaces with a space

    # 4. Final cleanup: lowercase and strip leading/trailing spaces
    text = text.lower().strip()
    
    return text

# Apply the cleaning function to the 'body' column
df["cleaned_email_body"] = df["body"].apply(clean_email_body)

# Drop the original, raw 'body' column as it's no longer needed
df = df.drop(columns="body")

print("\nDataFrame after basic cleaning:")
df.head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_30196\1898847667.py:11: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  soup = BeautifulSoup(html_text, 'html.parser')



DataFrame after basic cleaning:


,label,cleaned_email_body
0,0,date wed aug from ch...
1,0,martin a posted tassos papadopoulos the gree...
2,0,man threatens explosion in moscow thursday aug...
3,0,klez the virus that won t die already the mos...
4,0,in adding cream to spaghetti carbonara which ...


## Part 2: NLP Processing

- Downloaded the necessary NLTK packages (punkt, stopwords, wordnet).
- Initialized the `WordNetLemmatizer` and created a set of English `stopwords` for fast lookup.
- Defined a function (`preprocess_text_2`) that:
    - Tokenized the clean text (split it into a list of words).
    - Looped through the list and kept only words that were not in the stopword list.
    - Lemmatized each of those remaining words.
    - Returned `None` if the text became empty (e.g., it only contained stopwords).
    - Joined the final list of processed words back into a single string.
- Applied this function to the `cleaned_email_body` column.
- Stored the output in a new `final_text` column.
- Dropped the intermediate `cleaned_email_body` column.
- Dropped any rows that was null from the DataFrame.

### Download NLTK Assets

In [ ]:
# These only need to be downloaded once.
nltk.download('punkt')      # For the tokenizer
nltk.download('punkt_tab')  # Additional tokenizer resource
nltk.download('stopwords')  # For the list of stopwords
nltk.download('wordnet')    # For the lemmatizer
nltk.download('omw-1.4')    # Additional lemmatizer resource
nltk.download('words')      # For English vocabulary (less common, but good to have)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

### Initialize NLP Tools


In [ ]:
# Load stopwords into a set for faster lookup
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

print(f"\nStopwords loaded: {len(stop_words)}")
print(f"Tokenizer test: {word_tokenize('hello world!')}")

def preprocess_text_2(text, min_words=1):
    """
    Applies tokenization, stopword removal, and lemmatization.
    """
    # 1. Tokenize: Split text into a list of words
    tokens = word_tokenize(text)

    processed_tokens = []
    for word in tokens:
        # 2. Filter: Remove stopwords and keep only alphabetic words
        if word not in stop_words:
            # 3. Lemmatize: Reduce word to its root form
            processed_tokens.append(lemmatizer.lemmatize(word))

    # 4. Optional: Skip texts that become empty after processing
    # (e.g., an email that only contained stopwords)
    if len(processed_tokens) < min_words:
        return None # This will be dropped as NaN later

    # 5. Re-join the processed tokens into a single string
    return " ".join(processed_tokens)

# Apply the advanced NLP function
df['final_text'] = df['cleaned_email_body'].apply(preprocess_text_2)

# Drop the intermediate 'cleaned_email_body' column
df = df.drop(columns="cleaned_email_body")

# Drop any rows that became empty (NaN) during the NLP step
df = df.dropna()

print("\nMissing values per column after NLP processing:")
print(df.isna().sum())
df.head()


Stopwords loaded: 198
Tokenizer test: ['hello', 'world', '!']

Missing values per column after NLP processing:
label         0
final_text    0
dtype: int64


,label,final_text
0,0,date wed aug chris garrigues message id reprod...
1,0,martin posted tasso papadopoulos greek sculpto...
2,0,man threatens explosion moscow thursday august...
3,0,klez virus die already prolific virus ever kle...
4,0,adding cream spaghetti carbonara effect pasta ...


### DataFrame Formatting

In [100]:
# Rename 'label' to 'is_spam' for better readability
df.rename(columns={'label': 'is_spam'}, inplace=True)

# Reorder columns: feature (X) first, target (y) second
df = df[['final_text', 'is_spam']]

# Reset the DataFrame index after dropping rows
df = df.reset_index(drop=True)

print("\nFinal formatted DataFrame:")
df.head()


Final formatted DataFrame:


,final_text,is_spam
0,date wed aug chris garrigues message id reprod...,0
1,martin posted tasso papadopoulos greek sculpto...,0
2,man threatens explosion moscow thursday august...,0
3,klez virus die already prolific virus ever kle...,0
4,adding cream spaghetti carbonara effect pasta ...,0


In [101]:
# --- File Saving Utility ---

def get_filename_without_ext(file_dir):
    """
    Gets the filename (e.g., "Ling") without its extension
    from a given path (e.g., "../DS/Ling.csv").
    """
    # 1. Get the full filename (e.g., "Ling.csv")
    filename = os.path.basename(file_dir)
    # 2. Split the filename from its extension and return just the name
    filename_without_ext = os.path.splitext(filename)[0]
    return filename_without_ext

In [102]:
# --- Save Cleaned Data ---

# Define the output directory
output_dir = "../cleaned_DS/"

# Create the output directory if it doesn't already exist
os.makedirs(output_dir, exist_ok=True)

# Generate the new filename
filename_without_ext = get_filename_without_ext(file_dir)
output_path = f"{output_dir}{filename_without_ext}_cleaned.csv"

# Save the cleaned DataFrame to a new CSV, without the pandas index
df.to_csv(output_path, index=False)

print(f"\nSuccessfully cleaned and saved data to: {output_path}")
print(f"Total processed emails: {len(df)}")


Successfully cleaned and saved data to: ../cleaned_DS/Assassin_cleaned.csv
Total processed emails: 5800
